In [1]:
#Load collections of open source packages
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from collections import defaultdict
import pyodbc 
import pandas as pd
import numpy as np


In [45]:
import pyodbc 

#Connect to server and database
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DLRBIMSQL;'
                      'Database=ClarityIndex;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
#Build query asking database for the name of each room we have data on
query = """ SELECT [RoomModelId]
      ,[RoomName]
      FROM [ClarityIndex].[dbo].[PQ_RoomContents]
   group by [RoomModelId],[RoomName]
 """


In [46]:
#Build query asking database for the name of each room we have data on
query = """ SELECT [ItemId]
      ,[Name]
      FROM [ClarityIndex].[dbo].[PQ_RoomContents]
   group by [ItemId],[Name]
 """


In [47]:
#Send ask to database
RC = pd.read_sql_query(query,conn)

In [48]:
#Load custom functions cleaning and aggregating text

In [49]:
import nltk
def tokenlenfilter(tokenlist,minlen=2):
    tokens = [x for x in tokenlist if len(x)>minlen]
    return (tokens)

def lemmer(text):
    lemma_function = WordNetLemmatizer()

    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV

    return [lemma_function.lemmatize(token, tag_map[tag[0]]) for token, tag in pos_tag(text)]

def get_ngrams(series: pd.Series,n:int):
    ngrams = series.copy().str.split(' ').explode()
    ngrams1 = series.copy().str.split(' ').explode()
    for i in range(1,n):
        foo=ngrams1.groupby(level=0).shift(-i)
        ngrams=ngrams.str.cat(foo,sep=' ')
    ngrams = ngrams.dropna()
    return pd.DataFrame(ngrams)

def get_multigrams(series,n):
    grams=get_ngrams(series,1)
    if n==1:
        return grams
    else:
        for i in range(1,n+1):
            if i==1:
                continue
            grams=grams.append(get_ngrams(series,i))
    return grams

In [ ]:
#Throw away junk inside strings such as punctuation
RC['Name2']=RC['RoomName'].str.lower().str.replace('\d+','').str.replace('\+','').replace('[^\w\s]',' ').\
    str.lstrip().str.rstrip('-').str.rstrip()
#Throw away junk inside strings such as punctuation
RC['Name2']=RC['Name2'].str.replace('\.','').str.replace('\\',' ').str.replace('-',' ').str.replace('/',' ').str.replace("''",'')
#Idenity words within srings of words, throw away words with less than 2 letters, reduce words to root words, 
#join words back into one string of words
RC['Name3']=RC['Name2'].apply(nltk.word_tokenize).apply(tokenlenfilter).apply(lemmer).apply(' '.join).str.lstrip()
#Idenity words within srings of words,count the words, throw away words with a count below 75
RoomNameFrame=get_ngrams(RC['Name3'],1).value_counts()[get_ngrams(RC['Name3'],1).value_counts()>75].\
    reset_index().rename(columns={0:"Count"})
RoomNameFrame.head(60)

In [50]:
#Throw away junk inside strings such as punctuation
RC['Name2']=RC['RoomName'].str.lower().str.replace('\d+','').str.replace('\+','').replace('[^\w\s]',' ').\
    str.lstrip().str.rstrip('-').str.rstrip()

C:\Users\dmckenzie\Anaconda3\envs\Three Seven\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [51]:
#Throw away junk inside strings such as punctuation
RC['Name2']=RC['Name2'].str.replace('\.','').str.replace('\\',' ').str.replace('-',' ').str.replace('/',' ').str.replace("''",'')

C:\Users\dmckenzie\Anaconda3\envs\Three Seven\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
C:\Users\dmckenzie\Anaconda3\envs\Three Seven\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  


In [52]:
#Idenity words within srings of words, throw away words with less than 2 letters, reduce words to root words, 
#join words back into one string of words
RC['Name3']=RC['Name2'].apply(nltk.word_tokenize).apply(tokenlenfilter).apply(lemmer).apply(' '.join).str.lstrip()

In [53]:
#Idenity words within srings of words,count the words, throw away words with a count below 75
RoomNameFrame=get_ngrams(RC['Name3'],1).value_counts()[get_ngrams(RC['Name3'],1).value_counts()>75].\
    reset_index().rename(columns={0:"Count"})

In [54]:
RoomNameFrame.head(60)

,Name3,Count
0,room,4718
1,office,2732
2,storage,2684
3,,1660
4,classroom,1234
5,stair,1138
6,toilet,918
7,restroom,912
8,corridor,891
9,lobby,875


In [55]:
#Build query asking database for the number of times each item has been used within a room
query = """ SELECT [Name],count(*) as total
     FROM [ClarityIndex].[dbo].[PQ_RoomContents]
 Group BY [Name] 
order by count(*) DESC
 """
#Send ask to database
RCitem = pd.read_sql_query(query,conn)

In [56]:
#Throw away junk inside strings such as punctuation
RCitem['Name2']=RCitem['Name'].str.lower().str.replace('\d+','').str.replace('\+','').replace('[^\w\s]',' ').\
    str.lstrip().str.rstrip('-').str.rstrip()
#Throw away junk inside strings such as punctuation
RCitem['Name2']=RCitem['Name2'].str.replace('\.','').str.replace('\\',' ').str.replace('-',' ').str.replace('/',' ').str.replace('_',' ').str.replace("'",'')

C:\Users\dmckenzie\Anaconda3\envs\Three Seven\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
C:\Users\dmckenzie\Anaconda3\envs\Three Seven\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
C:\Users\dmckenzie\Anaconda3\envs\Three Seven\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  """


In [57]:
RCitem['Name2']

0           rectangular mullion
1               elbow   generic
2                       l angle
3             locker single dlr
4          door int hmframe dlr
                  ...          
56241                    mirror
56242         th floor x lights
56243          library casework
56244    academic locker infill
56245           a sensory table
Name: Name2, Length: 56246, dtype: object

In [58]:
#Get the names of Manufacturerers
query = """ 
SELECT Manufacturer,count(*) as totals
  FROM [ClarityIndex].[dbo].[PQ_RoomContents]
  where Manufacturer IS NOT NULL and Manufacturer<>' '
  GROUP BY Manufacturer
  Having count(*)>50
  order by count(*) DESC
"""
EVILNames = pd.read_sql_query(query,conn)

In [59]:
manufact=list(EVILNames['Manufacturer'].str.lower().str.replace('\d+','').str.replace('\+','').str.replace('[^\w\s]',' ').str.replace('_',' ').\
    str.lstrip().str.rstrip('-').str.rstrip().apply(nltk.word_tokenize).apply(' '.join))

C:\Users\dmckenzie\Anaconda3\envs\Three Seven\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [60]:
def tokennamefilter(tokenlist,minlen=2):
    tokens = [x for x in tokenlist if x not in manufact+['dlr','dlrj','dlrz','hightower','generic','zurn',
                                                               'campfire','turnstone',
                                                               'coalesse','mooreco','norvanivel',
                                                               'dividends horizon','knoll',
                                                               'steelcase','dlrjz','bobrick',
                                                               'dlrz','bradley','hermanmiller',
                                                               'herman','miller','bms']]
    return (tokens)

In [61]:
def tokenlenfilter(tokenlist,minlen=2):
    tokens = [x for x in tokenlist if len(x)>minlen]
    return (tokens)

In [62]:
#Idenity words within srings of words, throw away words with less than 2 letters, throw away manufacturer names,reduce words to root words, 
#join words back into one string of words
RCitem['Name3']=RCitem['Name2'].apply(nltk.word_tokenize).\
    apply(tokenlenfilter).\
    apply(tokennamefilter).\
    apply(lemmer).apply(' '.join).str.lstrip()

In [63]:
RCitem2=RCitem[RCitem['Name3']!=""]

In [64]:
RCitem2=RCitem2.groupby(['Name3'])['total'].sum().reset_index()

In [65]:
RCitem2['Key']=RCitem2['Name3']
RCitem2=RCitem2.set_index(['Key'])

In [66]:
get_ngrams(RCitem2['Name3'],1).\
    reset_index().\
    drop_duplicates(subset=['Key','Name3']).\
    set_index(['Key']).\
    join(RCitem2['total'])

,Name3,total
Key,,
aac,aac,133
aal,aal,5
aas high,aas,40
aas high,high,40
aash aboutastoolmetalbasehighheightwithoptionalfrontupholstery,aash,144
...,...,...
zuma series zboom desk,zuma,16
zuma series zboom desk,series,16
zuma series zboom desk,zboom,16


In [67]:
itemframe=get_ngrams(RCitem2['Name3'],1).\
    reset_index().\
    drop_duplicates(subset=['Key','Name3']).\
    set_index(['Key']).\
    join(RCitem2['total']).\
    groupby(['Name3'])['total'].sum().\
    reset_index()

In [68]:
def tokenlenfilter(tokenlist,minlen=2):
    tokens = [x for x in tokenlist if len(x)>minlen]
    return (tokens)

In [69]:
itemframe=itemframe[itemframe['Name3'].str.len()>3].query('total>7000')

In [70]:
itemframe.sort_values(['total'],ascending=False).head(60)

,Name3,total
8325,rectangular,489749
1729,chair,364742
2929,door,284332
6835,mullion,282784
3290,elbow,257655
1666,ceiling,253130
8300,recess,220698
8640,round,210291
11088,wall,205737
9207,single,186726


In [71]:
narratives=pd.read_csv('C:/Users/dmckenzie/OneDrive - DLR Group/Desktop/Project_narrative-export-visiondb.csv')

In [72]:
#narratives2=pd.read_csv('C:/Users/dmckenzie/OneDrive - DLR Group/Desktop/Project_narrative-export-visiondb.csv')
#narratives2.head(60)

In [73]:
narratives['CharCount']=narratives['Narrative'].astype(str).apply(len)

In [74]:
narratives

,WBS1,Narrative,CharCount
0,05-16101-00,Ryan Cameron and Michael Vander Ploeg pursued ...,306
1,05-16101-00,Ryan Cameron and Michael Vander Ploeg pursued ...,1412
2,05-21101-00,Design Achievement Emerging material technolog...,1622
3,10-00001-02,"Design Achievement - KANEKO, an Omaha based cu...",1855
4,10-00001-03,ADAM WELLS PERSONAL PROJECT Design Achieveme...,1391
...,...,...,...
2819,86-19101-01,Renderings and HNTE for Add Service-002,39
2820,86-19101-02,Test Fit and HNTE for Add Service-003,37
2821,86-19101-03,Renderings and ICON expenses HNTE for Add Serv...,53
2822,86-19101-04,Renderings and ICON expenses HNTE for Add Serv...,53


In [75]:
narratives['MaxChar']=narratives.groupby('WBS1')['CharCount'].transform('max')

In [76]:
narratives.head()

,WBS1,Narrative,CharCount,MaxChar
0,05-16101-00,Ryan Cameron and Michael Vander Ploeg pursued ...,306,1412
1,05-16101-00,Ryan Cameron and Michael Vander Ploeg pursued ...,1412,1412
2,05-21101-00,Design Achievement Emerging material technolog...,1622,1622
3,10-00001-02,"Design Achievement - KANEKO, an Omaha based cu...",1855,1855
4,10-00001-03,ADAM WELLS PERSONAL PROJECT Design Achieveme...,1391,1391


In [77]:
len(narratives)

2824

In [78]:
narratives=narratives[narratives['MaxChar']==narratives['CharCount']]

In [79]:
narratives=narratives.reset_index(drop=True)

In [80]:
narratives=narratives.drop_duplicates(['WBS1'])
narratives=narratives.reset_index(drop=True)

In [81]:
len(narratives)

2669

In [82]:
narratives['LastSentence']=narratives['Narrative'].astype(str).apply(nltk.sent_tokenize).str[-1]

In [109]:
servicesstring=['Acoustics', 'Architecture', 'Audiovisual', 'Bidding Support',
                'architectural design',
                'Bond Support','Bridging Documents','Civil Engineering', 'Conceptual Design',
                'Electrical Engineering', 'Environmental Graphics', 'Facility Assessments',
                'FAME - Facility Asset Management Evaluation','fame','Facility Asset Management Evaluation','It/Telecom',
                'Landscape Architecture', 'LEED Coordination', 'Lighting', 'Master Planning',
                'Mechanical Engineering', 'Security', 'Solar', 'Structural Engineering', 'Theater Technical', 
                'Visual Communication','Landscaping','bid support']

In [110]:
servicesstring = [item.lower() for item in servicesstring]

In [111]:
lemmaserviced=[' '.join(lemmer(nltk.word_tokenize(item))) for item in servicesstring]

In [112]:
lemmaserviced

['acoustic',
 'architecture',
 'audiovisual',
 'bidding support',
 'architectural design',
 'bond support',
 'bridging document',
 'civil engineering',
 'conceptual design',
 'electrical engineering',
 'environmental graphic',
 'facility assessment',
 'fame - facility asset management evaluation',
 'fame',
 'facility asset management evaluation',
 'it/telecom',
 'landscape architecture',
 'leed coordination',
 'light',
 'master planning',
 'mechanical engineering',
 'security',
 'solar',
 'structural engineering',
 'theater technical',
 'visual communication',
 'landscape',
 'bid support']

In [113]:
def servicefilter(tokenlist):
    tokens = [x for x in tokenlist if x in servicesstring+lemmaserviced or ' '.join(lemmer(nltk.word_tokenize(x))) in servicesstring+lemmaserviced]
    return (tokens)

In [114]:
def rationalizebigrams(tokenlist):
    tokens = [' '.join(x) for x in tokenlist]
    return (tokens)

In [115]:
narratives['UniService']=narratives['LastSentence'].str.lower().astype(str).apply(nltk.word_tokenize).apply(servicefilter)

In [116]:
narratives['DblService']=narratives['LastSentence'].str.lower().astype(str).\
    apply(nltk.word_tokenize).apply(nltk.bigrams).\
    apply(list).apply(rationalizebigrams).\
    apply(servicefilter)

In [117]:
narratives['ComboService']=narratives['UniService']+narratives['DblService']

In [118]:
len(narratives[narratives['ComboService'].apply(len)>0])/len(narratives)

0.4065192956163357

In [119]:
len(narratives[(narratives['Narrative'].astype(str).str.lower().str.contains('scope')) & 
               (narratives['ComboService'].apply(len)>0)])/len(narratives[narratives['Narrative'].astype(str).str.lower().str.contains('scope')])

0.5689958896065767

In [120]:
narratives[narratives['ComboService'].apply(len)>0]['ComboService'].apply(' '.join)

1                  conceptual design
3               visual communication
4               visual communication
5               visual communication
6               visual communication
                    ...             
2654                    architecture
2655                    architecture
2657                           light
2660                    architecture
2663    architecture master planning
Name: ComboService, Length: 1085, dtype: object

In [135]:
get_ngrams(narratives['LastSentence'].str.lower(),1).value_counts().head(60)

LastSentence 
and              2614
the              1813
dlr              1531
group            1443
provided         1193
design           1042
services.         871
of                841
to                799
a                 615
for               613
interior          558
architecture,     484
as                366
in                351
services          340
with              324
engineering,      301
is                300
engineering       288
architecture      286
design,           266
planning,         254
project           223
architectural     219
design.           214
through           210
new               198
planning          197
structural        192
this              180
construction      171
providing         164
integrated        154
will              153
building          151
existing          150
electrical        149
on                145
serve             144
firm              140
experience        137
master            136
resources         134
clients           

In [122]:
narratives['Narrative'].str.lower().str.contains('design achievement').sum()

1490

In [123]:
narratives['Narrative'].astype(str).apply(nltk.sent_tokenize).str[-1]

0       A secondary series of human-scaled vertical ma...
1       DLR Group is providing research leadership, an...
2       The crowning element of the structure is a bel...
3       The visual communication team provided brand v...
4       The visual communication team provided promoti...
                              ...                        
2664              Renderings and HNTE for Add Service-002
2665                Test Fit and HNTE for Add Service-003
2666    Renderings and ICON expenses HNTE for Add Serv...
2667    Renderings and ICON expenses HNTE for Add Serv...
2668                                    Hines T3 Test Fit
Name: Narrative, Length: 2669, dtype: object

In [124]:
from  rake_nltk import Rake  
r = Rake()
def rake_implement(x,r):
    r.extract_keywords_from_text(x)
    return r.get_ranked_phrases()


In [300]:
narratives['Narrative2']=narratives['Narrative'].str.replace('[^\w\s]','')

C:\Users\dmckenzie\Anaconda3\envs\Three Seven\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [301]:
narratives['Raked']=narratives['Narrative2'].astype(str).apply(lambda x: rake_implement(x,r))

In [302]:
NarExp=narratives['Narrative2'].astype(str).apply(lambda x: rake_implement(x,r)).explode().reset_index()

In [303]:
rooms=list(RoomNameFrame[(RoomNameFrame['Name3']!='') & 
                         (RoomNameFrame['Name3']!='dlr') &
                         (RoomNameFrame['Name3']!='test') &
                         (RoomNameFrame['Name3']!='group')]['Name3'])

In [304]:
len(rooms)

203

In [305]:
items=list(itemframe[(itemframe['Name3']!='') & (itemframe['Name3']!='dlr') & (itemframe['Name3']!='group')]['Name3'])
def tokennamefilter(tokenlist,minlen=2):
    tokens = [x for x in tokenlist if x in items+rooms]
    return (tokens)

In [306]:
NarExp

,index,Narrative2
0,0,living laboratory tom gallagher project princi...
1,0,michael vander ploeg pursued data streams inve...
2,0,mary miss city
3,0,iconic architectural lighting installation
4,0,design services watermarks phase1
...,...,...
138071,2666,renderings
138072,2667,icon expenses hnte
138073,2667,add service 13
138074,2667,renderings


In [307]:
NarExp['DesignScore']=NarExp['Narrative2'].astype(str).apply(nltk.word_tokenize).apply(tokennamefilter).apply(len)

In [308]:
NarExp['ChunkLen']=NarExp['Narrative2'].astype(str).apply(nltk.word_tokenize).apply(len)

In [309]:
NarExp['DesignRatio']=round(NarExp['DesignScore']/NarExp['ChunkLen'],2)

In [310]:
[element1+" "+element2 for element1, element2 in nltk.pos_tag(['fast','soft',
                                                  'hard','slow',
                                                  'hot','cold',
                                                  'fun','red','tall',
                                                  'short','small','best',
                                                  'worst','mean','hateful',
                                                 'pretty','ugly','loathesome',
                                                 'beatiful','marble']) if element2=='JJ' or 
 element2=='JJS' or element2=='JJR' or element2=='RB' or element2=='RBR' or element2=='RBS']

#[ele2 for ele1, ele2 in bigtuple if ele1 == "tuple1elem1"]

['fast RB',
 'soft JJ',
 'hard JJ',
 'hot JJ',
 'tall JJ',
 'short RB',
 'small JJ',
 'best JJS',
 'worst JJ',
 'hateful JJ',
 'pretty RB',
 'ugly RB',
 'loathesome JJ']

In [312]:
NarExp['InfoWords']=NarExp['Narrative2'].astype(str).\
    apply(nltk.word_tokenize).apply(lambda x: [element1 for element1, element2 in nltk.pos_tag(x) if element2=='JJ' or 
                                               element2=='JJS' or element2=='JJR' or element2=='RB' or element2=='RBR' or 
                                               element2=='RBS' or
                                               element2=='VB' or
                                               element2=='VBD' or
                                               element2=='VBG' or
                                               element2=='VBN' or
                                               element2=='VBP' or
                                               element2=='VBZ' or
                                               element1 in rooms+items])

In [263]:
'fire' in items

True

In [314]:
NarExp[((NarExp['InfoWords'].apply(len)/NarExp['ChunkLen'])>=.7) & 
       (NarExp['ChunkLen']>=2) & 
       (NarExp['DesignScore']>=1)]['Narrative2'].value_counts().head(35)

second floor             47
office space             39
focal point              38
first floor              34
natural light            33
south side               21
fitness center           21
mechanical electrical    21
north side               20
main entry               20
reception area           19
existing space           19
multipurpose room        19
national register        19
new space                15
east side                15
direct access            15
support space            14
meeting space            14
work included            14
top floor                13
conference center        13
new office               13
open office              13
retail space             12
conference room          12
west side                12
surrounding area         12
collaboration space      11
work includes            11
primary focus            11
front door               11
new entry                11
third floor              10
flexible space            9
Name: Narrative2, dt

In [363]:
goodphrases=list(NarExp[((NarExp['InfoWords'].apply(len)/NarExp['ChunkLen'])>=.7) & 
       (NarExp['ChunkLen']>=2) & 
        (NarExp['ChunkLen']<6) &
        (NarExp['Narrative2'].str.contains('sf')==False) &
        (NarExp['Narrative2'].str.contains('square feet')==False) &
        (NarExp['Narrative2'].str.contains('square foot')==False) &
        (NarExp['Narrative2'].str.contains('feet squared')==False) &                
       (NarExp['DesignScore']>=1)]['Narrative2'].drop_duplicates())

In [364]:
len(goodphrases)

6162

In [365]:
narratives['KeyPhrases']=narratives['Raked'].apply(lambda x:[i for i in x if i in goodphrases])

In [366]:
sum(narratives['KeyPhrases'].apply(len)>0)/len(narratives)

0.7287373548145373

In [367]:
pd.Series(goodphrases).to_csv('Desktop\KeyPhrases.csv')

In [368]:
narratives['KeyPhrases2']=narratives['KeyPhrases'].apply(', '.join)

In [369]:
narratives['ServiceTagsAndKeyPhrases2']=(narratives['ComboService']+narratives['KeyPhrases']).apply(', '.join)

In [370]:
narratives['ServiceTags']=narratives['ComboService'].apply(', '.join)

In [371]:
narrativesFinal=narratives[narratives['ServiceTagsAndKeyPhrases2']!=''][['WBS1','KeyPhrases2','ServiceTags','ServiceTagsAndKeyPhrases2']]

In [401]:
narratives['KeyPhrases'].astype(str).apply(len)

0        83
1       109
2       193
3        31
4        94
       ... 
2664      2
2665      2
2666      2
2667      2
2668      2
Name: KeyPhrases, Length: 2669, dtype: int64

In [402]:
narratives[narratives['KeyPhrases2']!='']

,WBS1,Narrative,CharCount,MaxChar,LastSentence,UniService,DblService,ComboService,Raked,Narrative2,KeyPhrases,KeyPhrases2,ServiceTagsAndKeyPhrases2,ServiceTags
0,05-16101-00,Ryan Cameron and Michael Vander Ploeg pursued ...,1412,1412,A secondary series of human-scaled vertical ma...,[],[],[],[living laboratory tom gallagher project princ...,Ryan Cameron and Michael Vander Ploeg pursued ...,"[iconic architectural lighting installation, s...","iconic architectural lighting installation, se...","iconic architectural lighting installation, se...",
1,05-21101-00,Design Achievement Emerging material technolog...,1622,1622,"DLR Group is providing research leadership, an...",[],[conceptual design],[conceptual design],[detailed cost analysis design considerations ...,Design Achievement Emerging material technolog...,"[us forestry service bringing almost, internat...","us forestry service bringing almost, internati...","conceptual design, us forestry service bringin...",conceptual design
2,10-00001-02,"Design Achievement - KANEKO, an Omaha based cu...",1855,1855,The crowning element of the structure is a bel...,[],[],[],"[orangepainted 2x4s pine wood slats 1x4, omaha...",Design Achievement KANEKO an Omaha based cult...,"[implemented using 12 mdf covered, given blank...","implemented using 12 mdf covered, given blank ...","implemented using 12 mdf covered, given blank ...",
3,10-00001-03,ADAM WELLS PERSONAL PROJECT Design Achieveme...,1391,1391,The visual communication team provided brand v...,[],[visual communication],[visual communication],[adam wells personal project design achievemen...,ADAM WELLS PERSONAL PROJECT Design Achieveme...,"[southern edge, main focus]","southern edge, main focus","visual communication, southern edge, main focus",visual communication
4,10-00001-04,Design Achievement - When the National Hospita...,1045,1045,The visual communication team provided promoti...,[],[visual communication],[visual communication],[dlr groups national design leaders personal e...,Design Achievement When the National Hospital...,"[soft cover saddlestiched book, national hospi...","soft cover saddlestiched book, national hospit...","visual communication, soft cover saddlestiched...",visual communication
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2659,84-19104-00,Design Achievement – Purple Dot Development as...,1711,1711,DLR Group served as the design architect and p...,[],[],[],[design achievement purple dot development ask...,Design Achievement Purple Dot Development ask...,"[also creates enough power, useable office spa...","also creates enough power, useable office spac...","also creates enough power, useable office spac...",
2660,84-19107-00,Design Achievement - This quarter of a million...,1983,1983,DLR Group is providing planning and architectu...,[architecture],[],[architecture],"[play sayani investments selected dlr group, i...",Design Achievement This quarter of a million ...,"[retail hospitality, work live, swimming pool,...","retail hospitality, work live, swimming pool, ...","architecture, retail hospitality, work live, s...",architecture
2661,84-19108-00,Design Achievement – Tracom Services’s vision ...,1354,1354,DLR Group served as the design architect and p...,[],[],[],[payment solutions company dlr groups design c...,Design Achievement Tracom Servicess vision wa...,"[swimming pool, oversailing canopy, central sp...","swimming pool, oversailing canopy, central space","swimming pool, oversailing canopy, central space",
2662,84-20101-00,Design Achievement – The Association of Member...,1272,1272,DLR Group served as the design architect and p...,[],[],[],"[even prospective tenants scope summary, incom...",Design Achievement The Association of Member ...,"[hotel leading away, hotel appear intertwined,...","hotel leading away, hotel appear intertwined, ...","hotel leading away, hotel appear intertwined, ...",


In [406]:
keyphrases=narratives[narratives['KeyPhrases2']!=''].\
    set_index('WBS1')[['KeyPhrases']].\
    explode('KeyPhrases').\
    reset_index()

In [419]:
keyphrases.to_csv('Desktop/KeyPhrasesExploded.csv')

In [405]:
servicetags=narratives[narratives['ServiceTags']!=''].\
    set_index('WBS1')[['ComboService']].\
    explode('ComboService').\
    reset_index()

In [413]:
servicetags['ComboService']=np.where(servicetags['ComboService']=='architectural design','architecture ',
        np.where(servicetags['ComboService']=='acoustic','acoustics',
                np.where(servicetags['ComboService']=='landscaped','landscape architecture',
                        np.where(servicetags['ComboService']=='landscaping','landscape architecture',
                                np.where(servicetags['ComboService']=='landscaped','landscape architecture',
                                        np.where(servicetags['ComboService']=='landscape','landscape architecture',
                                                np.where(servicetags['ComboService']=='light','lighting',
                                                        np.where(servicetags['ComboService']=='lights','lighting',
                                                                 servicetags['ComboService']))))))))

In [416]:
servicetags=servicetags[servicetags['ComboService']!='fame']

In [417]:
servicetags['ComboService'].value_counts()

architecture              790
structural engineering    167
lighting                  116
master planning           104
landscape architecture     98
electrical engineering     94
architecture               64
mechanical engineering     49
security                   37
audiovisual                36
acoustics                  34
civil engineering          17
conceptual design          16
theater technical          16
solar                      15
environmental graphics      9
facility assessment         6
bridging documents          5
it/telecom                  5
visual communication        5
environmental graphic       3
facility assessments        2
leed coordination           1
visual communications       1
Name: ComboService, dtype: int64

In [420]:
servicetags.to_csv('Desktop/ServiceTagsExploded.csv')

In [374]:
narrativesFinal.to_csv('Desktop/NarrativeKeyWords.csv')